In [1]:
#simple bert classifier


In [3]:
#  !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [1]:
import torch

# If there's a GPU available...
torch.cuda.is_available()    


True

In [2]:
import re
import matplotlib.pyplot as plt
import numpy as np
import nltk
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments


In [7]:
df=pd.read_json('yt_training_set.json')

df = pd.DataFrame(df)
# print(df)

#drop missing values
columns_to_dropna = ["title", "channel_title", "view_count", "tags", "description", "label"]

# dropna method drop rows with missing values 
filtered_data = df.dropna(subset=columns_to_dropna, how='any')

# print(filtered_data)
new_df=pd.DataFrame(filtered_data)



new_dataframe= pd.DataFrame({'title': new_df["title"].values,"channel_title": new_df["channel_title"].values,"view_count":new_df["view_count"].values,"tags":new_df["tags"].values,"description":new_df["description"].values,'label': new_df["label"].values})
# print(new_dataframe)
new_df=new_dataframe


# Convert the data to a pandas DataFrame
df_concat = pd.DataFrame(columns=["text", "label"])


for entry in new_df.index:
    
    title=new_df["title"].get(entry) 
    # print(title)
    
    channel_title=new_df["channel_title"].get(entry)
    
    tags=" ".join(new_df["tags"].get(entry ,[]))
    description=new_df["description"].get(entry)
    
    label=new_df["label"].get(entry)

    
    

    concat_text = f"{title} {channel_title} {tags} {description}"
    # concat_text=re.sub("(http\S+)", "", concat_text)
    
    new_row = pd.DataFrame({"text": [concat_text], "label": [label]})
    df_concat = pd.concat([df_concat, new_row], ignore_index=True)

# relabel the labels. 
#-1  replaced by 0, and  1 will be unchanged. 
df_concat['label'] = df_concat['label'].map({-1: 0, 1: 1})



In [6]:

# df_split: sentence and label
df_split = pd.DataFrame(columns=["sentence", "label"])

# Loop over the index of df_concat
for item in df_concat.index:
    
    
    text = df_concat["text"].get(item)
    label = df_concat["label"].get(item)
    
    # Split the text into sentences 
    sentences = nltk.sent_tokenize(text)
    
    # Loop over the sentences
    for sentence in sentences:
        
        df_split = pd.concat([df_split, pd.DataFrame({"sentence": [sentence], "label": [label]})], ignore_index=True)

# Print the df_split data frame
print(df_split)
print(df_split.shape)


                                               sentence label
0     The Second Book of Enoch (Human Voice, Read-Al...     0
1     By subscribing to my channel, along with likin...     0
2     SUBSCRIBE HERE: https://www.youtube.com/user/A...     0
3     This is not a work of fiction, but a genuine G...     0
4     The mere fact that it can be done at all is mo...     0
...                                                 ...   ...
7388  Box 2458\nAnaheim, CA 92814\n\n▬▬▬▬▬▬▬▬▬▬▬▬▬\n...     0
7389  Hurricane Dorian Scenarios: Breaking Down Stor...     0
7390  (Note: This video was recorded at 1 p.m. on Fr...     0
7391  For real-time updates on Hurricane Dorian, go ...     0
7392  NBC 6 provides news, weather, traffic and spor...     0

[7393 rows x 2 columns]
(7393, 2)


In [14]:

#! column name chnaged to sentence
X = df_split["sentence"].tolist()
y = df_split['label'].tolist()
# Split the data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(X,y, test_size=0.2)


In [15]:

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)




In [19]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    

# Create the datasets
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)




In [20]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,roc_auc_score

# Initialization of BERT 
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)  
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    auc = roc_auc_score(labels, predictions) 
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'roc-auc': auc # Added roc-auc
    }

# training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,  
)

# Create the trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


# Evaluate the model
eval_results = trainer.evaluate()

print(eval_results)
# {'eval_loss': 0.5771334767341614, 'eval_accuracy': 0.715, 'eval_f1': 0.6459627329192545,
#  'eval_precision': 0.7323943661971831, 'eval_recall': 0.5777777777777777, 'eval_runtime': 23.6281, 
# 'eval_samples_per_second': 8.465, 'eval_steps_per_second': 1.058, 'epoch': 3.0}

# {'eval_loss': 1.118485450744629, 'eval_accuracy': 0.775, 'eval_f1': 0.7513812154696132, 
# 'eval_precision': 0.8292682926829268, 'eval_recall': 0.6868686868686869, 'eval_roc-auc': 0.7741274127412741,
#  'eval_runtime': 23.5848, 'eval_samples_per_second': 8.48, 'eval_steps_per_second': 1.06, 'epoch': 10.0}
# 'eval_loss': 1.1889441013336182, 'eval_accuracy': 0.805, 'eval_f1': 0.7745664739884391, 
# 'eval_precision': 0.8589743589743589, 'eval_recall': 0.7052631578947368, 'eval_roc-auc': 0.800250626566416,
#  'eval_runtime': 23.4499, 'eval_samples_per_second': 8.529, 'eval_steps_per_second': 1.066, 'epoch': 20.0}

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/3700 [00:00<?, ?it/s]

{'loss': 0.4767, 'learning_rate': 1.72972972972973e-05, 'epoch': 2.7}
{'loss': 0.1376, 'learning_rate': 1.4594594594594596e-05, 'epoch': 5.41}
{'loss': 0.0614, 'learning_rate': 1.1891891891891894e-05, 'epoch': 8.11}
{'loss': 0.0407, 'learning_rate': 9.189189189189191e-06, 'epoch': 10.81}
{'loss': 0.0309, 'learning_rate': 6.486486486486487e-06, 'epoch': 13.51}
{'loss': 0.0271, 'learning_rate': 3.7837837837837844e-06, 'epoch': 16.22}
{'loss': 0.0229, 'learning_rate': 1.0810810810810812e-06, 'epoch': 18.92}
{'train_runtime': 56203.5335, 'train_samples_per_second': 2.104, 'train_steps_per_second': 0.066, 'train_loss': 0.1088173642674008, 'epoch': 20.0}


  0%|          | 0/185 [00:00<?, ?it/s]

{'eval_loss': 1.4830132722854614, 'eval_accuracy': 0.7701149425287356, 'eval_f1': 0.7546897546897547, 'eval_precision': 0.7805970149253731, 'eval_recall': 0.7304469273743017, 'eval_roc-auc': 0.7688931884577931, 'eval_runtime': 173.4484, 'eval_samples_per_second': 8.527, 'eval_steps_per_second': 1.067, 'epoch': 20.0}


In [21]:
# Save the model

# Specify the path
model_path = "models"
tokenizer_path = "models"

# Save the model and tokenizer

tokenizer.save_pretrained(tokenizer_path)

trainer.save_model(model_path)




In [22]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
# Evaluate the model
eval_results = trainer.evaluate()

print(eval_results)


  0%|          | 0/185 [00:00<?, ?it/s]

{'eval_loss': 1.4830132722854614, 'eval_accuracy': 0.7701149425287356, 'eval_f1': 0.7546897546897547, 'eval_precision': 0.7805970149253731, 'eval_recall': 0.7304469273743017, 'eval_roc-auc': 0.7688931884577931, 'eval_runtime': 176.0924, 'eval_samples_per_second': 8.399, 'eval_steps_per_second': 1.051, 'epoch': 20.0}


: 